In [13]:
#Importing libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import venn
import os 

%matplotlib inline

## pyUpset package for ...

orthofinder_dir_path    = '../Orthofinder-results-plot/apegreffi_corset_contracaecum_corset_asimplex_genome/OrthoFinder/Results_May21'
orthogroups_dir_path    = orthofinder_dir_path + '/Orthogroups'
orthologues_dir_path    = orthofinder_dir_path + '/Orthologues'
genomics_stats_dir_path = orthofinder_dir_path + '/Comparative_Genomics_Statistics'

# Getting the file names
stats_per_species       = genomics_stats_dir_path + '/Statistics_PerSpecies.tsv'
orthogroups_file        = orthogroups_dir_path + '/Orthogroups.tsv'

# Getting the organism names
organisms = ! head -n 1 $stats_per_species
organisms = organisms[0].split('\t')[1:]

In [29]:
# Take all orthogroups of each organism

df = pd.read_csv(orthogroups_file, sep='\t')

s = {}

for org in organisms:
    s[org] = set(df[df[org].notna()]['Orthogroup'])

venn.venn(s)